In [ ]:

import pandas as pd

import duckdb
from strava_duck_funcs_copy import *

# --- CONFIGURATION ---

FIT_FILES_FOLDER = '/Users/bac/Documents/PYTHON_PROJECTS/Strava_Analysis/strava_data_dumps/STRAVA+export_8029714/activities/'

#IMPORTANT_FIELDS = ['distance','enhanced_altitude','gps_accuracy','enhanced_speed','heart_rate', 'speed']

# Connect to DuckDB

DB_FILE = '../outputs/strava_test_2.duckdb'
con = duckdb.connect(DB_FILE)


In [ ]:
def create_tables():
    """Create activities and fit_logs tables (DuckDB compatible)."""
    con.execute("DROP TABLE IF EXISTS fit_logs")

    con.execute("""
        CREATE TABLE fit_logs (
            activity_id BIGINT,
            timestamp TIMESTAMP,
            long DOUBLE,
            lat DOUBLE,
            distance DOUBLE,
            enhanced_altitude DOUBLE,
                speed DOUBLE,
                gps_accuracy DOUBLE,
                heart_rate DOUBLE

                           )
    """)

    print("✅ Tables created fresh.")

In [ ]:

def bulk_insert_fit_logs(df_logs):
    """Bulk insert fit logs with auto-incremented log_ids."""

    if df_logs.empty:
        print("⚠️ No logs to insert.")
        return

    #next_id = get_next_id('fit_logs', 'log_id')
    #df_logs.insert(0, 'log_id', range(next_id, next_id + len(df_logs)))

    con.register('temp_fit_logs', df_logs)

    con.execute("""
        INSERT INTO fit_logs (activity_id, timestamp, long, lat, distance, enhanced_altitude, speed, gps_accuracy, heart_rate)
        SELECT activity_id, timestamp, long, lat, distance, enhanced_altitude, speed, gps_accuracy, heart_rate
        FROM temp_fit_logs
    """)

    print(f"✅ Inserted {len(df_logs)} logs starting at ID meh")

# def test_query():
#     """Query activities joined with fit_logs."""
#     df = con.execute("""
#         SELECT a.name, f.timestamp, f.field_name, f.field_value
#         FROM activities a
#         JOIN fit_logs f ON a.activity_id = f.activity_id
#         ORDER BY a.activity_id, f.timestamp
#         LIMIT 10
#     """).fetchdf()
#     print("\n🎯 Sample joined data:")
#     print(df)

Question now is what ID fields are needed in the fit logs

-- activity log is the key one, to join to some other setup, and otherwise, just some index...?


In [ ]:
fit_df = parse_fit_file(FIT_FILES_FOLDER)

In [ ]:
fit_df.columns

In [ ]:

# --- MAIN FLOW ---

if __name__ == "__main__":
    create_tables()
    bulk_insert_fit_logs(fit_df)
    # test_query()
    print("\n🏁 All done!")


In [ ]:
def test_query():
    """Query activities joined with fit_logs."""
    df = con.execute("""
        SELECT distinct activity_type
        FROM activities
        JOIN fit_logs f ON a.activity_id = f.activity_id
        ORDER BY a.activity_id, f.timestamp
        LIMIT 10
    """).fetchdf()
    print("\n🎯 Sample joined data:")
    print(df)